# Saving and Restoring Models
Once you have trained your model, why should we save its parameters?
1. you can come back to it ewhenever you want.
2. using it in anotehr program.
3. comparing it to other models, and so on.

Moreover, you can prevent from losing your parameters by saving checkpoints at regular intervals during training. So, how? Just create a `Saver` node at the end of the construction phase (after all variable nodes are created). then, in the execution phase, just call its `save()` method whenever you want to save the model, passing it the session and path of the checkpoint file.

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn import preprocessing

# Adding an extra bias input feature (x0 = 1) to all training instances
housing = fetch_california_housing()
m, n = housing.data.shape # (20640, 8)
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # (20640, 9)

# Normalize the feature vector: scaled_housing_data_plus_bias
scaler =preprocessing.StandardScaler().fit(housing.data)
scaled_housing_data = scaler.transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [2]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * batch_index)
    batch_mark = np.random.choice(m, batch_size)
    X_batch = scaled_housing_data_plus_bias[batch_mark]
    y_batch = housing.target.reshape(-1,1)[batch_mark]
    return X_batch, y_batch

In [3]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") # random_uniform() generating a tensor containing random values
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()
# saver = tf.train.Saver({"weights": theta})

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: # checkpoint every 100 epochs
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

            

# restoring your model
# with tf.Sesson() as sess:
    # saver.restore(sess, "./tmp/my_model_final.ckpt") 

    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")    

# Visualizing the Graph and Training Curves Using TensorBoard
The TensorBoard nicely display interactive visulizations of some taining stats you feed in your webv browser (e.g., learning curves).
1. writes the graph definition and some training stats - the training error (MSE).
2. need to use a different log directory every time you run your program: include a timestamp in the log directory name

In [4]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir =  "./{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") # random_uniform() generating a tensor containing random values
y_pred = tf.matmul(X, theta, name="prediction")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()
# saver = tf.train.Saver({"weights": theta})

# creates a node in the graph that will evaluate the MSE value 
# and write it to a TensorBoard-compatible bianry log string called a summary
mse_summary = tf.summary.scalar('MSE', mse)
# creates a FileWriter that you will use to write summaries to logfiles in the log directory.
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: # checkpoint every 100 epochs
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0: # Avoid logging training stats at every single training step
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

# restoring your model
# with tf.Sesson() as sess:
    # saver.restore(sess, "./tmp/my_model_final.ckpt") 

    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")
file_writer.close()